In [1]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import math
import glob
from IPython.display import clear_output
from pip import main



In [2]:
def eulard(A,B):
    return np.sqrt(sum(np.power((A-B),2)))

# def eular3d(A,B):
#     return np.sqrt(sum(np.power((A-B),2)))

In [3]:
def show_img(img, bigger=False):
    if bigger:
        plt.figure(figsize=(15,15))
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(image_rgb)
    plt.show()

def mouse_handler(event, x, y, flags, data):
    if event == cv2.EVENT_LBUTTONDOWN:
        # 標記點位置
        cv2.circle(data['img'], (x,y), 3, (0,0,255), 5, 16) 

        # 改變顯示 window 的內容
        cv2.imshow("Image", data['img'])
        
        # 顯示 (x,y) 並儲存到 list中
        print("get points: (x, y) = ({}, {})".format(x, y))
        data['points'].append((x,y))

def get_points(im):
    # 建立 data dict, img:存放圖片, points:存放點
    data = {}
    data['img'] = im.copy()
    data['points'] = []
    
    # 建立一個 window
    cv2.namedWindow("Image", 0)
    
    # 改變 window 成為適當圖片大小
    h, w, dim = im.shape
    print("Img height, width: ({}, {})".format(h, w))
    cv2.resizeWindow("Image", w, h)
        
    # 顯示圖片在 window 中
    cv2.imshow('Image',im)
    
    # 利用滑鼠回傳值，資料皆保存於 data dict中
    cv2.setMouseCallback("Image", mouse_handler, data)
    
    # 等待按下任意鍵，藉由 OpenCV 內建函數釋放資源
    cv2.waitKey()
    cv2.destroyAllWindows()
    
    # 回傳點 list
    return data['points']

In [4]:
def caculation(data):
    totalPA = []
    totalPB = []
    tanglePA = []
    tanglePB = []

    for i ,element in enumerate(data):
    #     print (i,element)
    #     print(element[0])
        A = np.array([element[0][0],element[0][1]])
        B = np.array([element[1][0],element[1][1]])
        P = np.array([element[2][0],element[2][1]])
        distAB = eulard(A,B) # pixel for units
        distPA = eulard(P,A)
        distPB = eulard(P,B)
        perPA = distPA/distAB
        perPB = distPB/distAB
        anglePA = np.arccos(np.dot((B-A),(P-A))/(distAB*distPA))
        anglePB = np.arccos(np.dot((B-A),(P-B))/(distAB*distPB))

    #     print(type(perPA))
        totalPA.append(perPA)
        totalPB.append(perPB)
        tanglePA.append(anglePA)
        tanglePB.append(anglePB)



    print(totalPA)
    print(totalPB)
    avgPA = np.mean(totalPA)
    avgPB = np.mean(totalPB)
    stdPA = np.std(totalPA)
    stdPB = np.std(totalPB)
    avganglePA = np.mean(tanglePA)
    avganglePB = np.mean(tanglePB)
    print(avgPA,avgPB)
    print(stdPA,stdPB)
    print('the avganglePA is',avganglePA,"(/rad)")
    # print(avganglePB)
    print(tanglePA)
    # print(tanglePB)


    #     print(dist)
    #     dist = element
    
    return avgPA,avgPB,avganglePA



In [5]:
# def drawfaceline(nose,ear,avgPA,avgPB):
def drawfaceline(avgPA,avgPB,avganglePA):
    
    
    nose = np.array([232,176]) #have to be measurement before
    ear = np.array([398,287]) #have to be measurement before
#     path = './Face_Portrait/val/fusion_1.jpg'
    path = './por3.jpg'
    img = cv2.imread(path)
    img = cv2.resize(img, (640,480), interpolation=cv2.INTER_AREA)
    cv2.line(img, nose, ear, (0, 0, 255), 3)
    
    X = eulard(nose,ear)
    near = (ear-nose)*avgPA
    
    if near[0] <= 0:
        avganglePA = -avganglePA
    p = (near[0]*np.cos(avganglePA) - near[1]*np.sin(avganglePA) , near[0]*np.sin(avganglePA) + near[1]*np.cos(avganglePA))
    p = p + nose
    p = p.astype(int)
    print(p) # the position is global position
    cv2.line(img, nose, p, (0, 255, 0), 2)    
    
    

    cv2.imshow('windows',img)
    
    
#     key = cv2.waitKey(0) & 0xFF
#     if key == ord('q'):
#         cv2.destroyAllWindows()
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

In [74]:
def transform(point):
    new_point = np.array([point[2],point[1],-point[0]])
    return new_point

def mapping_on_plate():
    original_point = np.array([[310,200,280],[300,300,300],[300,400,290]]) # mm for units 
    new_camera = np.array([-800,0,800])
    face_plate_vector = np.cross((original_point[0]-original_point[1]),(original_point[0]-original_point[2]))
    portrait_point = np.array([[500,0,800],[400,20,790],[420,10,790]])
    # this is the original detected position from newcam (nose,ear,nastropy) , so we have to do transformation
#     print('portrait',portrait_point)
    
    new_portrait_point = np.empty([3,3])
    print(new_portrait_point)
    for i,point in enumerate (portrait_point):
        new_portrait_point[i] = transform(point) + new_camera
        print(i,point)
    print(new_portrait_point)
    print('face' , face_plate_vector)
    
    map_point = new_portrait_point + face_plate_vector * ( (np.dot(face_plate_vector,new_portrait_point[0]) + np.dot(face_plate_vector,portrait_point[0]))/ eulard(face_plate_vector,0))
    print(map_point)

In [75]:
mapping_on_plate()

[[  0.   0. 300.]
 [-10.  20. 400.]
 [-10.  10. 380.]]
0 [500   0 800]
1 [400  20 790]
2 [420  10 790]
[[  0.   0. 300.]
 [-10.  20. 400.]
 [-10.  10. 380.]]
face [-3000  -100 -1000]
[[2465344.21083995   82178.14036133  822081.40361332]
 [2465334.21083995   82198.14036133  822181.40361332]
 [2465334.21083995   82188.14036133  822161.40361332]]


In [6]:
if __name__ == "__main__":
    data = {}
    data["pointsall"] = []
    
#Read the destination image
    for i in range(1,11): # last number will not count
        img = './Face_Portrait/out/fusion_' + str(i) + '.jpg'
        img_dst = cv2.imread(img)

        # print("Click on the screen and press any key for end process")
        points  = get_points(img_dst)

        print("\npoints list of fusion_" + str(i) +':')
        # pointsall = pointsall.extend(points)
        data["pointsall"].append(points)
        print(points)
        

    print(data["pointsall"])
    avgPA,avgPB,avganglePA = caculation(data["pointsall"])


Img height, width: (640, 480)
get points: (x, y) = (191, 187)
get points: (x, y) = (317, 248)
get points: (x, y) = (274, 234)

points list of fusion_1:
[(191, 187), (317, 248), (274, 234)]
Img height, width: (640, 480)
get points: (x, y) = (193, 189)
get points: (x, y) = (305, 260)
get points: (x, y) = (260, 230)

points list of fusion_2:
[(193, 189), (305, 260), (260, 230)]
Img height, width: (640, 480)
get points: (x, y) = (178, 183)
get points: (x, y) = (277, 250)
get points: (x, y) = (248, 241)

points list of fusion_3:
[(178, 183), (277, 250), (248, 241)]
Img height, width: (640, 480)
get points: (x, y) = (231, 190)
get points: (x, y) = (338, 259)
get points: (x, y) = (294, 233)

points list of fusion_4:
[(231, 190), (338, 259), (294, 233)]
Img height, width: (640, 480)
get points: (x, y) = (250, 194)
get points: (x, y) = (352, 249)
get points: (x, y) = (317, 228)

points list of fusion_5:
[(250, 194), (352, 249), (317, 228)]
Img height, width: (640, 480)
get points: (x, y) = (164

In [ ]:
drawfaceline(avgPA,avgPB,avganglePA)

[339 259]
